<a href="https://colab.research.google.com/github/garrer/NTERSOL_DE/blob/main/Copy_of_SparkTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Code required to get Spark environment

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 48.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=70d675de186e5d6e5a57f357d2ef5a14b0d7a2e8a79fb83ad75a21c578e043e7
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [6]:
raw_df = spark.read.csv('sample_data/data-engineer-interview-data.csv', header=True)

In [7]:
raw_df.createOrReplaceTempView("TitleData")

In [8]:
spark.sql("select * from TitleData limit 10")

DataFrame[DocumentNumber: string, DocumentDate: string, DocumentType: string, RefersToDocumentNumber: string, RefersToDocumentYear: string, Remarks: string]

In [9]:
raw_df.show(10)

+--------------+------------+------------+----------------------+--------------------+------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|     Remarks|
+--------------+------------+------------+----------------------+--------------------+------------+
|            27|   8/16/2021|           C|                  null|                null|        null|
|             1|   2/15/2021|           T|                     8|                2020|2020 0008-00|
|            67|   10/9/2020|           A|                  null|                null|        null|
|           157|   10/9/2020|           A|                  null|                null|        null|
|           189|   10/9/2020|           J|                  null|                null|        null|
|           250|   10/9/2020|           R|                  2016|                  98|        null|
|            87|    8/7/2020|           R|                  1992|                  97|        null|


In [15]:
spark.sql("""
                select *, CAST ((DocumentNumber+split(DocumentDate,'/')[2]) AS INT) as document_unique_Id,  
                    CAST ( COALESCE(RefersToDocumentNumber, 0)+ COALESCE (RefersToDocumentYear, 0) AS INT) as document_ref_unique_Id                   
                    from TitleData
                    """).createOrReplaceTempView("derived")

In [17]:
spark.sql("""select * from derived limit 10""").show(10)

+--------------+------------+------------+----------------------+--------------------+------------+------------------+----------------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|     Remarks|document_unique_Id|document_ref_unique_Id|
+--------------+------------+------------+----------------------+--------------------+------------+------------------+----------------------+
|            27|   8/16/2021|           C|                  null|                null|        null|              2048|                     0|
|             1|   2/15/2021|           T|                     8|                2020|2020 0008-00|              2022|                  2028|
|            67|   10/9/2020|           A|                  null|                null|        null|              2087|                     0|
|           157|   10/9/2020|           A|                  null|                null|        null|              2177|                     0|
|     

In [29]:
spark.sql ("""select t1.*, t2.DocumentType as ReferenceType           
                 from derived t1 INNER JOIN derived t2 on t1.document_ref_unique_Id = t2.document_unique_Id"""            
               ).createOrReplaceTempView("derived_final")

In [30]:
spark.sql("""select * from derived_final limit 10""").show(10)

+--------------+------------+------------+----------------------+--------------------+------------+------------------+----------------------+-------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|     Remarks|document_unique_Id|document_ref_unique_Id|ReferenceType|
+--------------+------------+------------+----------------------+--------------------+------------+------------------+----------------------+-------------+
|             1|   2/15/2021|           T|                     8|                2020|2020 0008-00|              2022|                  2028|            B|
|           250|   10/9/2020|           R|                  2016|                  98|        null|              2270|                  2114|            C|
|           250|   10/9/2020|           R|                  2016|                  98|        null|              2270|                  2114|            A|
|           250|   10/9/2020|           R|                  2016

In [23]:
%%markdown

* Documents of type J (a "judgment" document) no longer apply if a document of type T (a "termination" document) refers to it
* Documents of any other type no longer apply if a document of type R (a "release" document) refers to it




* Documents of type J (a "judgment" document) no longer apply if a document of type T (a "termination" document) refers to it
* Documents of any other type no longer apply if a document of type R (a "release" document) refers to it



In [33]:
spark.sql ("""
            
            SELECT * FROM derived_final t1 WHERE t1.document_unique_Id NOT IN (
            SELECT  document_unique_Id  FROM derived_final t1 where (lower (t1.DocumentType) == 't' and
            lower (t1.ReferenceType) == 'j') OR (lower (t1.ReferenceType) == 'r')) 
                
            """).createOrReplaceTempView("TitleDataFinal")

In [34]:
#test case: Check if TitleDataFinal has any document type J referencing to document type T
spark.sql (""" select * from TitleDataFinal where upper(DocumentType)=='T' and upper(ReferenceType)=='J' """).show(10)

+--------------+------------+------------+----------------------+--------------------+-------+------------------+----------------------+-------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|Remarks|document_unique_Id|document_ref_unique_Id|ReferenceType|
+--------------+------------+------------+----------------------+--------------------+-------+------------------+----------------------+-------------+
+--------------+------------+------------+----------------------+--------------------+-------+------------------+----------------------+-------------+



In [35]:
#test case: Check if TitleDataFinal has any record referencing document type R
spark.sql (""" select * from TitleDataFinal where upper(ReferenceType)=='R' """).show(10)

+--------------+------------+------------+----------------------+--------------------+-------+------------------+----------------------+-------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|Remarks|document_unique_Id|document_ref_unique_Id|ReferenceType|
+--------------+------------+------------+----------------------+--------------------+-------+------------------+----------------------+-------------+
+--------------+------------+------------+----------------------+--------------------+-------+------------------+----------------------+-------------+

